In [1]:
! pip install category-encoders

In [9]:
import numpy as np # linear algebra
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
import pandas as pd

import category_encoders as ce

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
pd.set_option('display.max_columns', 200)

In [3]:
path = ""

In [4]:
data = pd.concat([pd.read_csv(path + 'Train.csv'), pd.read_csv(path + 'train_2.csv')], ignore_index=True)
data = data.drop_duplicates()
test = pd.read_csv(path + 'Test.csv')

In [5]:
data.head(2)

,ID,TypeLogmt_1,TypeLogmt_2,TypeLogmt_3,H08_Impute,H09_Impute,TAILLE_MENAGE,H17A,H17B,H17C,H17D,H17E,H17F,H17G,H17H,H17I,H17J,H17Y,H18A,H18B,H18C,H18D,H18E,H18F,H18G,H18H,H18I,H18J,H18Y,H20A,H20B,H20C,H20D,H20E,H20Y,H21A,H21B,H21C,H21D,H21Y,Connexion,BoxLabel,,.1,.2,.3,.4,.5,.6,.7,.8,.9,.10,.11,.12,.13,.14,.15,.16,.17,.18,.19,.20,.21,.22,.23,.24,.25,.26,.27,.28,.29,.30,.31,.32,.33,.34,.35,.36,.37,.38,.39,.40,.41,.42,.43,.44,.45,.46,.47,.48,.49,.50,.51,.52,.53,.54,.55,.56,.57,...,.3901,.3902,.3903,.3904,.3905,.3906,.3907,.3908,.3909,.3910,.3911,.3912,.3913,.3914,.3915,.3916,.3917,.3918,.3919,.3920,.3921,.3922,.3923,.3924,.3925,.3926,.3927,.3928,.3929,.3930,.3931,.3932,.3933,.3934,.3935,.3936,.3937,.3938,.3939,.3940,.3941,.3942,.3943,.3944,.3945,.3946,.3947,.3948,.3949,.3950,.3951,.3952,.3953,.3954,.3955,.3956,.3957,.3958,.3959,.3960,.3961,.3962,.3963,.3964,.3965,.3966,.3967,.3968,.3969,.3970,.3971,.3972,.3973,.3974,.3975,.3976,.3977,.3978,.3979,.3980,.3981,.3982,.3983,.3984,.3985,.3986,.3987,.3988,.3989,.3990,.3991,.3992,.3993,.3994,.3995,.3996,.3997,.3998,.3999,Accès internet
0,ID_88M0FMHZAM,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,1.0,Non,Oui,Non,Non,Non,Non,Non,Non,Non,Non,Non,Oui,Non,Non,Non,Non,Non,Non,Non,Non,Non,Non,Oui,Oui,Non,Non,Non,Non,Non,Non,Non,Non,Oui,GVA,Box0,0.110878,0.364513,0.082156,0.050989,0.180205,0.440745,0.091101,0.350169,0.012656,0.414666,0.115399,0.195980,0.336563,0.078171,0.100897,0.045542,0.913928,0.018699,0.380138,0.607484,0.277331,0.138209,0.276000,0.066287,0.258161,0.288994,0.682467,0.254205,0.464460,0.478994,0.392388,0.255803,0.084414,0.229634,0.532075,0.666907,0.188527,0.046141,0.028404,0.159643,0.070380,0.254873,0.323476,0.569911,0.236242,0.331764,0.083024,0.019692,0.084888,0.229847,0.502781,0.055822,0.166818,0.664789,0.175114,0.614555,0.308945,1.400921,...,0.229371,0.016024,0.936691,0.257711,0.168477,0.637111,0.159609,0.079461,0.703852,0.104156,0.010313,0.098713,0.000021,0.716649,0.133922,0.615847,0.501898,0.011709,0.566245,0.048341,1.477248,1.140575,0.617236,0.192597,0.637621,0.120499,0.396786,0.454566,1.264199,0.430409,0.016467,1.351908,0.070340,0.240572,1.641209,0.274469,0.291744,0.988008,0.011058,0.459556,0.000022,0.550851,0.095509,0.441678,1.813182,0.001026,0.000226,0.002365,0.023680,0.017586,1.731053,0.033532,0.215995,0.0,0.713431,0.003149,0.039846,0.830917,0.979955,0.014376,1.043260,0.037076,0.012557,0.0000,0.083004,0.112426,1.166675,0.317973,1.451176,0.132251,0.000000,0.265171,0.459284,0.049367,0.378637,0.013094,0.023059,0.000620,0.078775,0.673005,0.195031,0.327815,0.291608,0.744023,0.170152,0.503821,0.066751,0.000000,0.060418,0.183824,0.025202,0.235426,0.251377,0.125899,0.834979,1.055083,0.503122,0.761878,0.018388,0
1,ID_8BRUVGMYT4,Logement semi-moderne,Plusieurs logement,Logement � un niveau (plain-pied),Electricit� (CEET) compteur dans la concession,Gaz,4.0,Non,Oui,Non,Non,Non,Non,Non,Non,Non,Non,Non,Oui,Non,Non,Non,Oui,Non,Non,Non,Non,Non,Non,Oui,Non,Oui,Non,Oui,Non,Non,Non,Non,Non,Oui,GVA,Box0,0.070467,0.174473,0.077245,0.009509,0.065423,0.181117,0.089368,0.513371,0.007979,0.136733,0.148871,0.215091,0.277986,0.097039,0.108392,0.027131,0.401973,0.002417,0.191969,0.224114,0.172935,0.127753,0.355751,0.037032,0.119602,0.137235,0.309211,0.110607,0.257442,0.191642,0.176879,0.124613,0.036168,0.155968,0.531614,0.302581,0.294130,0.084554,0.006434,0.065791,0.019145,0.101057,0.238687,0.325745,0.348762,0.192260,0.056421,0.033568,0.157190,0.222881,0.286392,0.038866,0.174491,0.406858,0.251796,0.439162,0.157752,1.030446,...,0.268409,0.039308,1.131682,0.641069,0.259756,0.542906,0.685960,0.159447,0.062619,0.177584,0.172911,0.166974,0.008849,0.424695,0.850363,0.351184,0.398521,0.194348,0.346673,0.394468,0.220056,0.056125,0.630001,0.222548,0.415653,0.361412,0.433209,0.594335,0.162098,0.668560,0.293882,0.377199,0.335916,0.393318,0.002772,0.725495,0.297718,0.878449,0.151743,0.388457,0.008659,0.561902,0.120055,0.364930

In [6]:
data = data.drop(columns='ID')
data.shape

(30863, 4042)

In [7]:
categorical_cols = data.select_dtypes(include=['object', 'category']).columns
numeric_cols = data.select_dtypes(include=['int64', 'float64']).columns
numeric_cols = numeric_cols.drop('Accès internet')


# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns='Accès internet'), data['Accès internet'], test_size=0.3, random_state=42)

In [ ]:
"""def add_squared_features(X):
    return pd.DataFrame({col+'_squared': X[col]**2 for col in X.columns}, index=X.index)

# Feature engineering pour les colonnes numériques (valeurs et carrés)
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler()),
    ('squared', FunctionTransformer(add_squared_features, validate=False))
])

# OneHotEncoding pour les variables catégorielles
onehot_transformer = OneHotEncoder(handle_unknown='ignore')

# Target Encoding pour les variables catégorielles
target_encoder = ce.TargetEncoder(cols=categorical_cols)

# Combinaison des transformations numériques et catégorielles
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('onehot', onehot_transformer, categorical_cols),
        ('passthrough', 'passthrough', categorical_cols)  # 'passthrough' car TargetEncoder sera appliqué après
    ])

# Pipeline final incluant la Target Encoding après le préprocesseur
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('target_encoder', target_encoder),
    ('classifier', RandomForestClassifier(random_state=42))
])"""

In [10]:
class BinarizeNumerical(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.means_ = np.nanmean(X, axis=0)
        return self
    
    def transform(self, X, y=None):
        return (X > self.means_).astype(int)

numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('binarizer', BinarizeNumerical())
])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_cols),
        ('cat', categorical_pipeline, categorical_cols)
    ]
)

In [16]:
# Création du pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=102))
])

In [ ]:
### Assez long
param_grid = {
    'classifier__n_estimators': [100, 200, 300],
    'classifier__max_depth': [10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4,7,]
}

grid_search = GridSearchCV(model, param_grid, cv=10, scoring='roc_auc', n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
best_params = grid_search.best_params_
print(f'Best parameters: {best_params}')

In [ ]:
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

In [ ]:
# Faire des prédictions sur l'ensemble de test pour évaluer le modèle
y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Probabilités des classes positives

# Évaluer le modèle
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)

print(f'Accuracy: {accuracy:.2f}')
print(f'AUC: {auc:.2f}')

In [17]:
model.fit(data.drop(columns='Accès internet'), data['Accès internet'])

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('binarizer',
                                                                   BinarizeNumerical())]),
                                                  Index(['TAILLE_MENAGE', ' ', ' .1', ' .2', ' .3', ' .4', ' .5', ' .6', ' .7',
       ' .8',
       ...
       ' .3990', ' .3991', ' .3992', ' .3993', ' .3994', ' .3995', ' .3996',
       ' .3997', ' .3998', ' .3999'],
      dtype='object', length=4001)),
                                                 ('cat',
                                                  Pipeline(step...
                                                  Index(['TypeLogmt_1', 'TypeLogmt_2', 'TypeLogmt_3', 'H08_Impute', 'H09_Impute',
       'H17A', 'H17B', 'H17C', 'H17D', 'H17E', 'H17F', 'H17G', 'H17H', 'H17I',
       'H17J', 'H17Y', 'H18A', 'H18B', 'H18C', 'H18D', 'H18E', 'H18F', 'H18G',
       'H18H', 'H18I', 'H18J', 'H18Y', 'H20A', 'H20B', 'H20C', 'H20D', 'H20E',
       'H20Y', 'H21A', 'H21B', 'H21C', 'H21D', 'H21Y', 'Connexion',
       'BoxLabel'],
      dtype='object'))])),
                ('classifier', RandomForestClassifier())])

In [18]:
ID = test.ID
X_test_processed = model.named_steps['preprocessor'].transform(test.drop(columns='ID'))
test_predictions = model.named_steps['classifier'].predict_proba(X_test_processed)[:, 1]

submission = pd.DataFrame({'ID': ID, 'Target': test_predictions})
submission.to_csv('submission_190624_3.csv', index=False)